## Initial Setup

In [ ]:
# Establish search path and import local utilities module
""" This includes: 
    get_data_path(), get_csv_file(data_path, file_name), 
    display_file_summary(data_frame, name=""), display_file_stats(data_frame, name="")
"""
import sys
sys.path.append("/Users/stuartgow/GitHub/Python_Snippets/Utilities")
import SGfilehandling as files

# Import typically used packages
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Import plotting packages
import matplotlib.pyplot as plt
plt.style.use('ggplot')
#plt.style.use('default')

In [3]:
# CSV File Data Load to a Pandas dataframe & Initial Examination
# The 'Data' folder one level above the project 
data_path = files.get_data_path()   
file_name="wallacecommunications.csv"

df = files.get_csv_file(data_path, file_name=file_name)

print(f'File Contents. {file_name}')
display(df.head())
files.display_file_summary(df, name=file_name)
files.display_file_stats(df, name=file_name)

File Contents. wallacecommunications.csv


,ID,town,country,age,job,married,education,arrears,current_balance,housing,has_tv_package,last_contact,conn_tr,last_contact_this_campaign_day,last_contact_this_campaign_month,this_campaign,days_since_last_contact_previous_campaign,contacted_during_previous_campaign,outcome_previous_campaign,new_contract_this_campaign
0,32390233,Bristol,UK,39,management,married,tertiary,no,-1.90,no,yes,unknown,4,11,jun,8,-1,0,unknown,yes
1,41226029,Basildon,UK,53,housemaid,married,primary,no,3.62,no,no,cellular,1,28,jul,2,-1,0,unknown,yes
2,85498057,Blackburn,UK,79,retired,divorced,primary,no,27.87,no,no,telephone,3,13,jul,1,377,1,success,yes
3,76539627,Oldham,UK,43,services,single,secondary,no,5.50,yes,no,cellular,3,13,may,2,-1,0,unknown,no
4,76290986,Middlesbrough,UK,35,technician,single,secondary,no,3.44,no,no,cellular,2,27,aug,16,-1,0,unknown,yes


File Details. wallacecommunications.csv


,Count,Missing,Empty,Unique,Top,Freq,Type,String,Int,Float,Bool,List,Other
ID,50662,0,0,50644,80354425,2,int64,0,50662,0,0,0,0
town,50662,0,0,101,London,3241,object,50662,0,0,0,0,0
country,50662,0,0,5,UK,50657,object,50662,0,0,0,0,0
age,50662,0,0,76,32,2273,int64,0,50662,0,0,0,0
job,50662,0,0,12,management,10849,object,50662,0,0,0,0,0
married,50662,0,0,3,married,30045,object,50662,0,0,0,0,0
education,50662,0,0,4,secondary,25779,object,50662,0,0,0,0,0
arrears,50662,0,0,2,no,49794,object,50662,0,0,0,0,0
current_balance,50662,0,0,6928,0.0,3836,float64,0,0,50662,0,0,0
housing,50662,0,0,2,yes,27250,object,50662,0,0,0,0,0


File Stats For Numerics. wallacecommunications.csv


,count,mean,std,min,25%,50%,75%,max
ID,50662.00,55028712.75,25985959.78,10000726.00,32499637.25,55011272.00,77488092.25,99999515.00
age,50662.00,40.98,10.92,18.00,32.00,39.00,48.00,95.00
current_balance,50662.00,13.99,30.90,-80.19,0.82,4.72,14.95,984.17
conn_tr,50662.00,3.01,1.41,1.00,2.00,3.00,4.00,5.00
last_contact_this_campaign_day,50662.00,15.77,8.35,1.00,8.00,16.00,21.00,31.00
this_campaign,50662.00,2.71,3.05,1.00,1.00,2.00,3.00,63.00
days_since_last_contact_previous_campaign,50662.00,42.64,102.24,-1.00,-1.00,-1.00,-1.00,871.00
contacted_during_previous_campaign,50662.00,0.64,2.36,0.00,0.00,0.00,0.00,275.00


## Data Cleaning

- ID - 18 values are not unique?
- Countries ex UK?
- has tv package - 3 values
- conn_tr - understand this grouping
- last contact - combine month and day into a date field?
- this_campaign - are there some unusually high values for number of contacts?
- contacted_during_previous_campaign - ditto
- outcome_previous_campaign - 4 different outcome categories?



In [53]:
customers_cleaned = df.copy()

In [54]:
# Data Cleaning - Variable by Variable

# ID - Identify duplicates - But leave as won't impact the predictions
duplicated = customers_cleaned[customers_cleaned.duplicated(keep=False, subset=['ID'])]
print(f'Duplicates: {duplicated.shape[0]}')
#display(duplicated.iloc[:, :7].sort_values(by='ID'))

# Town - ok

# Country - Some incorrect countries, change so All are UK
display(customers_cleaned[customers_cleaned['country'] != 'UK'])
customers_cleaned['country'] = 'UK'
display(customers_cleaned[customers_cleaned['country'] != 'UK'])


Duplicates: 36


,ID,town,country,age,job,married,education,arrears,current_balance,housing,has_tv_package,last_contact,conn_tr,last_contact_this_campaign_day,last_contact_this_campaign_month,this_campaign,days_since_last_contact_previous_campaign,contacted_during_previous_campaign,outcome_previous_campaign,new_contract_this_campaign
632,79806939,Basildon,France,39,management,married,tertiary,no,5.62,no,no,cellular,3,13,sep,2,-1,0,unknown,no
658,20895885,Aberdeen,France,49,technician,divorced,secondary,no,-2.36,yes,no,unknown,1,26,may,2,-1,0,unknown,no
747,14855363,Norwich,Germany,37,management,married,tertiary,no,6.30,no,no,cellular,2,30,apr,1,87,1,success,yes
1338,74486719,Sale,USA,26,technician,single,secondary,no,12.31,yes,no,cellular,2,3,jun,2,96,5,success,yes
6746,54594566,Birmingham,Portugal,50,housemaid,married,tertiary,no,52.60,yes,no,cellular,1,24,jul,2,-1,0,unknown,no


,ID,town,country,age,job,married,education,arrears,current_balance,housing,has_tv_package,last_contact,conn_tr,last_contact_this_campaign_day,last_contact_this_campaign_month,this_campaign,days_since_last_contact_previous_campaign,contacted_during_previous_campaign,outcome_previous_campaign,new_contract_this_campaign


## Training & Validation & Test Split

In [ ]:
# Select the desired features and the target variable
targetName="new_contract_this_campaign"
featureNames=df.columns[df.columns != targetName]

X = df[featureNames]
y = df[targetName]

?? K-fold validation ... validation and test data split?

In [7]:
# Simple split of training and testing data, 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Data Preprocessing

In [ ]:
# Categorical values - Convert to numerical and encode
- town
- country
- job
- married
- education
- arrears
- housing
- has_tv_package
- last_contact
- outcome_previous_campaign
- new_contract_this_campaign .... target

